# Name: Mohammad Javad Noroozi
# Student Number: 99102434
# HW3

### import the necessary library

In [1]:
import json
import datetime
from collections import Counter


In [2]:
users = [{'1':'yek'}, {'1':'yek'}, {'2':'do'}]

In [5]:
users = [('1', 'yek'), ('1', 'yek'), ('2', 'do')]


In [7]:
counts = Counter(users)
users = counts.most_common()

In [11]:
a,b = zip(*users)

In [12]:
a

(('1', 'yek'), ('2', 'do'))

In [2]:
# For measuring the total running time of this notebook
start_notebook_t = datetime.datetime.now()
print(start_notebook_t)

2024-02-02 03:26:39.181372


In [3]:
import findspark
findspark.find()
findspark.init()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("HW1") \
    .master("local[*]") \
    .getOrCreate()

sc=spark.sparkContext


24/02/02 03:26:40 WARN Utils: Your hostname, javad-IdeaPad-Gaming-3-15IAH7 resolves to a loopback address: 127.0.1.1; using 192.168.137.8 instead (on interface enp48s0)
24/02/02 03:26:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/02 03:26:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Read data

In [4]:
tweets_rdd = sc.textFile("twitter_data_v2.jsonl")


### Downsampling (If your RAM memory is low)

In [5]:
# tweets_rdd = tweets_rdd.sample(False, 0.1, seed=42)


### Show a sample of tweets

In [6]:
tweets_rdd.takeSample(False, 1, 19)

['{"in_reply_to_status_id_str": null, "in_reply_to_user_id_str": null, "in_reply_to_status_id": null, "in_reply_to_user_id": null, "in_reply_to_screen_name": null, "truncated": false, "is_quote_status": false, "retweet_count": 3, "reply_count": 0, "quote_count": 0, "favorite_count": 0, "favorited": false, "retweeted": true, "possibly_sensitive": null, "lang": "fa", "geo": null, "view_count": 0, "tweet_type": "retweeted", "emojis": null, "text": "مردک وطن\u200cفروش بدبخت، ترانه آفتابکاران جنگل مال چی بوده که خوندن؟! موسیقی اصیل ایرانی بوده؟! یا سرود تروریست\u200cهای چریک کمونیست بوده؟!\\nاشکال نداره، شما به خزعبل گویی ادامه بدید. خوبیش اینه که مردم دارن کم کم متوجه میشن که شنا دگوری\u200cهای خائن وطن\u200cفروش چه جماعتی هستید.\\n\\n#ننگ_بر_سه_فاسد_ملا_چپی_مجاهد", "user": {"id": "315689091", "id_str": "315689091", "created_at": 1307868062, "name": "The Nerd", "screen_name": "NewbieBadASS", "location": "خیابون هفده شهریور ", "url": "https://t.co/ajpmxoYoTe", "description": "A supporter of

### Show size of data (No. tweets)

In [7]:
tweets_rdd.count()

4175860

In [8]:
users = tweets_rdd.map(lambda x: json.loads(x)["user"].get('id')).collect()

In [9]:
'''
you must choose user id in this section.
you can change # in `counts.most_common()[#][0]` for this purpose.
'''
counts = Counter(users)
q_uid = counts.most_common()[50][0]
print('query user id:', q_uid)

query user id: 1726657633799569408


### Define Parameters for the Primary Rating.

In [10]:
# Define Parameters
owner_s = 1

# following parameters should not be equal to owner_s
rep_to_s = 0.2
rep_from_s = 0.9

ret_to_s = 0.5
ret_from_s = 1.5

qut_to_s = 0.6
qut_from_s = 1.5


### Define functions for primary and secondary ratings

In [11]:
def to_user_tid_score(tid, uid, type, origin_tid, origin_uid):
    ''' This function used for calculate primary rating matrix.
    
    This function get some information of a tweet and Returns
    the score that two users (the user who posted this tweet
    and the user who posted the original tweet and has now
    been replied to, retweeted, or quoted by this tweet)
    give to these two tweets FOR EACH USER.

    The output is a list of tuples, where each tuple contains
    a user ID and a list of tweets that have been rated (along
    with the value of the rating).
    Output format: [((u_i, ((t_j, s_ij),)) ...]
    '''
    
    origin_tid = [i for i in origin_tid if i is not None]
    origin_uid = [i for i in origin_uid if i is not None]

    if not origin_tid or not origin_uid:
        return [(uid, [(tid, owner_s)])]

    to_s = rep_to_s if type == "replied" else\
           ret_to_s if type == "retweeted" else\
           qut_to_s if type == "quoted" else\
           0
    from_s = rep_from_s if type == "replied" else\
             ret_from_s if type == "retweeted" else\
             qut_from_s if type == "quoted" else\
             0

    return [(uid, [(tid, owner_s), (origin_tid[0], from_s)]),
            (origin_uid[0], [(tid, to_s)]),
           ]


def to_tid_user_score(tid, uid, type, origin_tid, origin_uid):
    ''' This function used for secondary rating.

    This function get some information of a tweet and Returns
    the score that two users (the user who posted this tweet
    and the user who posted the original tweet and has now
    been replied to, retweeted, or quoted by this tweet)
    give to these two tweets FOR EACH TWEET.

    The output is a list of tuples, where each tuple contains
    a tweet ID and a tuple list of users rate to it (along with
    the value of the rating).
    Output format: [(tid_i, ((u_j, s_ij), ...)), ...]
    '''
    if uid != q_uid and origin_uid != q_uid:
        return []
    
    origin_tid = [i for i in origin_tid if i is not None]
    origin_uid = [i for i in origin_uid if i is not None]

    if not origin_tid or not origin_uid:
        return [(tid, ((uid, owner_s),))]

    to_s = rep_to_s if type == "replied" else\
           ret_to_s if type == "retweeted" else\
           qut_to_s if type == "quoted" else\
           0
    from_s = rep_from_s if type == "replied" else\
             ret_from_s if type == "retweeted" else\
             qut_from_s if type == "quoted" else\
             0

    return [(tid, ((uid, owner_s), (origin_uid[0], to_s),)),
            (origin_tid[0], ((uid, from_s),)),
           ]


def extract_tuples(tweet_info, func):
    '''
    This function extract some information of tweet and send it
    to one of the following funcions and return their results:
    - to_user_tid_score
    - to_tid_user_score
    '''
    return func(
        tweet_info.get('id'),
        tweet_info["user"].get('id'),
        tweet_info.get('tweet_type'),
        [
            tweet_info.get('in_reply_to_status_id_str', None),
            tweet_info.get('quoted_status', {}).get('id'),
            tweet_info.get('retweeted_status', {}).get('id'),
        ],
        [
            tweet_info.get('in_reply_to_user_id_str', None),
            tweet_info.get('quoted_status', {}).get('user', {}).get('id'),
            tweet_info.get('retweeted_status', {}).get('user', {}).get('id'),
        ],
    )


def extract_relations(x):
    '''
    This function extracts relationships between users who interacted
    with this tweet.
    
    x contains the tweet_id (tid) and a list of tuples. Each tuple
    contains a user_id (u) and its interaction with the tweet (score
    given to the tweet).
    x: (tid, [(u1, s1), ...])

    The output is a list that each element of it, contains the user_id
    (u1) and a list of tuples. Each tuple contains a tweet_id (tid), 
    second user_id that has relation to u1 (called u2), and the score
    given by each of these two users to this tweet (s1 and s2 respectively)
    output: 
    [(u1, [(tid, u2, s1, s2),
            (tid, u3, s1, s3),
            ...
           ]),
     (u2, [(tid, u1, s2, s1),
            (tid, u3, s2, s3),
            ...
                  ]),
     ...
    ]
    
    '''

    tid, user_score = x

    for (u1, s1) in user_score:
        if u1 == extract_relations.q_uid:
            u1_relations = []
            for (u2, s2) in user_score:
                if u1 != u2 and s1 > rep_to_s:
                    u1_relations.append((tid, u2, s1, s2))
            if u1_relations:
                return [(u1, sorted(u1_relations, key=lambda x: -x[2]*x[3])[:min(100, len(u1_relations))])]
        
    return [(None,[])]


def find_user_favorites(x, user_scores):
    ''' 
        user_scores: {ui: [(t_j, s_ij), ...]}
        x: (u1, ((tid, uj, s1, sj), ...))
    '''
    th = 0.05
    u, relations = x
    favorite = {}
    if relations:
        for (tid, uj, s1, s2) in relations:
            favorite[tid] = favorite.get(tid, 0) + s1
            for (t, s) in user_scores.get(uj, []):
                favorite[t] = favorite.get(t, 0) + s1*s2*s
    if favorite:
        max_favorite = max(favorite.values())
        th = 0.05 * max_favorite
        while len(favorite) > 100:
            favorite = {t:i for t,i in favorite.items() if i > th}
            th *= 1.1
        return (u, favorite)
    return (u,{})


### Calculating Primary rating Matrix

In [12]:
start_calculating_matrix = datetime.datetime.now()
print(start_calculating_matrix)


2024-02-02 03:27:51.050887


In [13]:
# [((u_i, ((t_j, s_ij),)), ...]
user_tid_score_rdd = tweets_rdd.flatMap(
    lambda x: extract_tuples(json.loads(x), func=to_user_tid_score))

# [(u_i, ((t_j, s_ij), ...)), ...]
user_tids_scores_rdd = user_tid_score_rdd.reduceByKey(
    lambda x, y: x if not x.extend(y) else x)

user_scores_list = user_tids_scores_rdd.collect()

user_scores_dict = dict(user_scores_list)


### Extracting the query user's favorite tweets id


In [14]:
start_extracting_tweets_id_t = datetime.datetime.now()
print(start_extracting_tweets_id_t)

2024-02-02 03:28:29.677275


In [15]:
# [(tid, [(u, s)]), ...]
tid_user_score_rdd = tweets_rdd.flatMap(lambda x: extract_tuples(json.loads(x), func=to_tid_user_score))

# [(tid, [(u, s), ...]), ...]
tid_users_scores_rdd = tid_user_score_rdd.reduceByKey(lambda x, y: x + y)

# [(q_uid, [(tid, u2, s1, s2)])]
extract_relations.q_uid = q_uid
user_relation_rdd = tid_users_scores_rdd.flatMap(extract_relations)\
                                         .filter(lambda x: x[1])

# [(q_uid, [(tid, u2, s1, s2), ...])]
user_relations_rdd = user_relation_rdd.reduceByKey(lambda x, y: x if not x.extend(y) else x)


In [16]:
# Take query user's relations in ram
user_relations = user_relations_rdd.take(1)[0]

# Find the ID of the user's favorite tweets
user_favorites = find_user_favorites(x=user_relations, user_scores=user_scores_dict)



In [17]:
user_favorites

('1726657633799569408',
 {'1735046002074112361': 197.10000000000002,
  '1735544709492166984': 324.29999999999995,
  '1735566464436875406': 109.60000000000002,
  '1735312336808763439': 198.15,
  '1735525089020359052': 257.3,
  '1735365389419176136': 182.25,
  '1735159525215199507': 114.5,
  '1735660869005025381': 205.4,
  '1735358691241582790': 78.9,
  '1735619676606996647': 87.25,
  '1735541697801539687': 252.35,
  '1735741892493504797': 191.95000000000002,
  '1735324458104025433': 67.55,
  '1736039059980509252': 73.05,
  '1736080376219554071': 139.9,
  '1736038955571662982': 89.55,
  '1735862892669386955': 71.75,
  '1735601169777451322': 180.9,
  '1736011315812983209': 94.2,
  '1736063860208021928': 100.75,
  '1736035559162712134': 83.1,
  '1735939488029814818': 82.25,
  '1735981539328995706': 96.35000000000001,
  '1735975098991976653': 64.19999999999999,
  '1735881101401305327': 122.75,
  '1735744858709692683': 167.85,
  '1736012732397183174': 118.0,
  '1735631011323511083': 146.75,


In [18]:
if not user_favorites:
    print('Please choose another User ID, this user has no favorite tweet in this database')

## Extracte and Show query user's tweets and the query user's favorite tweets information

In [19]:
start_extract_and_show_tweets_t = datetime.datetime.now()
print(start_extract_and_show_tweets_t)


2024-02-02 03:29:03.172966


In [20]:
necessary_keys = ['tweet_type', 'user', 'id','text', 'name', 'screen_name', 
                  'in_reply_to_status_id_str', 'in_reply_to_user_id_str',
                  'quoted_status', 'retweeted_status','profile_image_url',
                 ]


In [21]:
favorites_tweets = tweets_rdd.filter(lambda x: json.loads(x)['id'] in user_favorites[1].keys() or
                                               json.loads(x)['user']['id'] == q_uid)\
                             .map(lambda x: dict(filter(lambda y: y[0] in necessary_keys, json.loads(x).items())))\
                             .collect()
favorites_tweets2 = favorites_tweets.copy()


### Define functions needed to separate tweets based on tweet_type and display tweets appropriately

In [22]:
def extract_user_origin_tweets(tweets, uid, rem=True):
    origin_tweets = []
    for t in tweets:
        if t['tweet_type'] == 'generated':
            if t.get('user',{}).get('id','0') == uid:
                origin_tweets.append(t)
    if rem:
        for t in origin_tweets:
            tweets.remove(t)
    return origin_tweets


def extract_user_replied_tweets(tweets, uid, rem=True):
    replied_tweets = []
    for t in tweets:
        if t['tweet_type'] == 'replied':
            if t.get('user',{}).get('id','0') == uid:
                replied_tweets.append(t)
    if rem:
        for t in replied_tweets:
            tweets.remove(t)
    return replied_tweets


def extract_user_quoted_tweets(tweets, uid, rem=True):
    quoted_tweets = []
    for t in tweets:
        if t['tweet_type'] == 'quoted':
            if t.get('user',{}).get('id','0') == uid:
                quoted_tweets.append(t)
    if rem:
        for t in quoted_tweets:
            tweets.remove(t)
    return quoted_tweets


def extract_user_retweeted_tweets(tweets, uid, rem=True):
    retweeted_tweets = []
    for t in tweets:
        if t['tweet_type'] == 'retweeted':
            if t.get('user',{}).get('id','0') == uid:
                retweeted_tweets.append(t)
    if rem:
        for t in retweeted_tweets:
            tweets.remove(t)
    return retweeted_tweets


def extract_replied_to_user_tweets(tweets, uid, rem=True):
    replied_tweets = []
    for t in tweets:
        if t['tweet_type'] == 'replied':
            if t.get('in_reply_to_user_id_str', '0') == uid:
                replied_tweets.append(t)
    if rem:
        for t in replied_tweets:
            tweets.remove(t)
    return replied_tweets


def extract_quoted_user_tweets(tweets, uid, rem=True):
    quoted_tweets = []
    for t in tweets:
        if t['tweet_type'] == 'quoted':
            if t.get('quoted_status', {}).get('user', {}).get('id', '0') == uid:
                quoted_tweets.append(t)
    if rem:
        for t in quoted_tweets:
            tweets.remove(t)
    return quoted_tweets


def extract_retweeted_user_tweets(tweets, uid, rem=True):
    retweeted_tweets = []
    for t in tweets:
        if t['tweet_type'] == 'retweeted':
            if t.get('retweeted_status', {}).get('user', {}).get('id', '0') == uid:
                retweeted_tweets.append(t)
    if rem:
        for t in retweeted_tweets:
            tweets.remove(t)
    return retweeted_tweets


def extract_explore_tweets(tweets):
    return tweets


def show_tweets(tweets, tweets_type):
    print('-'*100)
    print(f'Number of {tweets_type}:', len(tweets), '\n')
    ts = [t['text'].replace('\u200c', '').split('\n') for t in tweets]
    for t in ts:
        print('-'*100)
        for line in t:
            print(line)
    print('-'*100)


In [23]:
uid = q_uid
favorites_tweets = favorites_tweets2.copy()
user_origin_tweets = extract_user_origin_tweets(favorites_tweets, uid)
user_quoted_tweets  = extract_user_quoted_tweets(favorites_tweets, uid)
user_retweeted_tweets  = extract_user_retweeted_tweets(favorites_tweets, uid)
user_replied_tweets  = extract_user_replied_tweets(favorites_tweets, uid)
replied_to_user_tweets  = extract_replied_to_user_tweets(favorites_tweets, uid)
quoted_user_tweets  = extract_quoted_user_tweets(favorites_tweets, uid)
retweeted_user_tweets  = extract_retweeted_user_tweets(favorites_tweets, uid)
explore_tweets = extract_explore_tweets(favorites_tweets)


In [24]:
show_tweets(user_origin_tweets, 'user_origin_tweets')

----------------------------------------------------------------------------------------------------
Number of user_origin_tweets: 0 

----------------------------------------------------------------------------------------------------


In [25]:
show_tweets(user_quoted_tweets, 'user_quoted_tweets')


----------------------------------------------------------------------------------------------------
Number of user_quoted_tweets: 15 

----------------------------------------------------------------------------------------------------
شهدا مصداق کامل عالم سلوک و مسافر هفتاقلیم عشقاند، کسانی که از قید جان گذشتند و رفتند تا ثابت کنند عاشقاند..
#وداع_با_لالهها 
#فرزندان_فاطمه
----------------------------------------------------------------------------------------------------
هنوز هم چشم بهراهند؛ پدران و مادرانی که روزها و شبها را سر کرده و امروز نیز با ورود هر شهیدی با پای فرتوت و چشمان اشکبار راهی تشییع شهدایی میشوند که بوی فرزندشان را استشمام کنند، شهدایی که گاهی بینام و گاهی هویدا میآیند.
#وداع_با_لالهها 
#فرزندان_فاطمه
----------------------------------------------------------------------------------------------------
هنوز هم چشم بهراهند؛ پدران و مادرانی که روزها و شبها را سر کرده و امروز نیز با ورود هر شهیدی با پای فرتوت و چشمان اشکبار راهی تشییع شهدایی میشوند که بوی فرزندشان را اس

In [26]:
show_tweets(user_retweeted_tweets, 'user_retweeted_tweets')


----------------------------------------------------------------------------------------------------
Number of user_retweeted_tweets: 1096 

----------------------------------------------------------------------------------------------------
پلاک نداشت ...
اصلاً هیچ نوشته ای نداشت...
امیدوار بودم روی زیرپیراهنی اش اسمش رو نوشته باشه.
نوشته بود :
اگر برای خداست بگذار گمنام بمانم
#وداع_با_لالهها 
#فرزندان_فاطمه https://t.co/cALw8zSORN
----------------------------------------------------------------------------------------------------
شهید گمنام ؛
یعنی شهیدی که می توانست عقب بیاید ...
اما ماند
#وداع_با_لالهها 
#فرزندان_فاطمه https://t.co/jumojfdmmv
----------------------------------------------------------------------------------------------------
مهدی فاطمه میآید
  مزار مادرمون پیدا میشود #أللَّھُـمَ_عـجِّـلْ_لِوَلیِڪْ_ألْـفَـرَج
#وداع_با_لالهها
#فرزندان_فاطمه https://t.co/1rwSchzEV3
----------------------------------------------------------------------------------------------------
میرس

In [27]:
show_tweets(user_replied_tweets, 'user_replied_tweets')


----------------------------------------------------------------------------------------------------
Number of user_replied_tweets: 0 

----------------------------------------------------------------------------------------------------


In [28]:
show_tweets(explore_tweets, 'explore_tweets')


----------------------------------------------------------------------------------------------------
Number of explore_tweets: 96 

----------------------------------------------------------------------------------------------------
قسمتی از نامه دختر شهید تجلایی 
پدر جان ای کاش مزاری داشتی تا شبهای جمعه به سراغت بیایم و درد دل کنم !نه ،چه می گویم !؟مگر میشود عشق را خاک کرد ؟مگر سرخی عشق با سیاهی خاک در هم می آمیزد؟هیهات !ما کجا و افلاک کجا ؟!
#وداع_با_لالهها
#فرزندان_فاطمه https://t.co/u67H5unt41
----------------------------------------------------------------------------------------------------
ای شقایق های آتش گرفته
دل خونین ما شقایقی است که داغ شهادت شمارا برخورد دارد. آیا آن روز نیز خواهد رسید که بلبلی دیگر در وصف ما سرود شهادت بسراید؟
  شهید اهل قلم سید مرتضی آوینی
#وداع_با_لالهها
#فرزندان_فاطمه https://t.co/oexanIw8ER
----------------------------------------------------------------------------------------------------
این روزهاعطر #فرزندان_فاطمه  فضای آلوده ی شهرم را قابل استشمام

In [29]:
end_tweet_recommending_t = datetime.datetime.now()
print(end_tweet_recommending_t)
print("The total time spent finding and displaying the user's favorite tweets:",
      end_tweet_recommending_t-start_calculating_matrix)


2024-02-02 03:29:36.687742
The total time spent finding and displaying the user's favorite tweets: 0:01:45.636855


# User Recommender 

In [30]:
start_user_recommending_t = datetime.datetime.now()
print(start_user_recommending_t)


2024-02-02 03:29:36.692903


In [31]:
def extract_users(x):
    tweet_info = json.loads(x)

    uids = [tweet_info["user"].get('id')]

    origin_uid = [tweet_info.get('in_reply_to_user_id_str', None),
                  tweet_info.get('quoted_status', {}).get('user', {}).get('id'),
                  tweet_info.get('retweeted_status', {}).get('user', {}).get('id'),
                 ]

    origin_uid = [i for i in origin_uid if i is not None]

    if origin_uid:
        uids.append(origin_uid[0])
    
    return uids

In [32]:
recommended_users = tweets_rdd.filter(lambda x: json.loads(x)['id'] in user_favorites[1].keys() or
                                               json.loads(x)['user']['id'] == q_uid)\
                              .flatMap(extract_users).collect()
                            


In [33]:
counts = Counter(recommended_users)
mcru = counts.most_common(20)  # most_common_recommended_users
mcru_id, repeats = zip(*mcru[1:])
mcru[1:]

[('1720551038975930369', 24),
 ('1555619939154804742', 24),
 ('1708791760720195584', 20),
 ('1660730673085161475', 18),
 ('1628027425249538051', 17),
 ('1381298273084960784', 17),
 ('1655227383270588418', 16),
 ('1665036207217147904', 16),
 ('1638186511165161474', 16),
 ('1723740609066758144', 15),
 ('1723735777387876352', 13),
 ('1704008195675443200', 13),
 ('1724855378788458496', 12),
 ('1633423628011290625', 12),
 ('1381639321346842628', 12),
 ('1675286065081466880', 11),
 ('1697793813052284928', 11),
 ('1339531627215642624', 10),
 ('1553764179559038976', 10)]

### Show Tweets of recommended Users

In [34]:
start_extract_recommended_users_tweets_t = datetime.datetime.now()
print(start_extract_recommended_users_tweets_t)


2024-02-02 03:30:10.208363


In [35]:
rec_users_tweets = tweets_rdd.filter(lambda x: json.loads(x)['user']['id'] in mcru_id)\
                            .map(lambda x: dict(filter(lambda y: y[0] in necessary_keys, json.loads(x).items())))\
                            .collect()



In [36]:
end_extract_recommended_users_tweets_t = datetime.datetime.now()
print(end_extract_recommended_users_tweets_t)



2024-02-02 03:30:34.872250


#### Tweets of Query User

In [37]:
favorites_tweets = favorites_tweets2.copy()
print('','#'*100,'\n', 'Tweets of Query User\n', '#'*100,'\n')
show_tweets(extract_user_origin_tweets(favorites_tweets, q_uid, rem=False), 'user_origin_tweets')
show_tweets(extract_user_quoted_tweets(favorites_tweets, q_uid, rem=False), 'user_quoted_tweets')
show_tweets(extract_user_retweeted_tweets(favorites_tweets, q_uid, rem=False), 'user_retweeted_tweets')
show_tweets(extract_user_replied_tweets(favorites_tweets, q_uid, rem=False), 'user_replied_tweets')

 #################################################################################################### 
 Tweets of Query User
 #################################################################################################### 

----------------------------------------------------------------------------------------------------
Number of user_origin_tweets: 0 

----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Number of user_quoted_tweets: 15 

----------------------------------------------------------------------------------------------------
شهدا مصداق کامل عالم سلوک و مسافر هفتاقلیم عشقاند، کسانی که از قید جان گذشتند و رفتند تا ثابت کنند عاشقاند..
#وداع_با_لالهها 
#فرزندان_فاطمه
----------------------------------------------------------------------------------------------------
هنوز هم چشم بهراهند؛ پدران و مادرانی که روزها و شبها را 

#### Tweets of Recommended User

In [38]:
for (recommended_uid, rep) in mcru[1:]:
    print('','#'*100,'\n', f'User ID: {recommended_uid}, repetitions: {rep}\n', '#'*100,'\n')
    show_tweets(extract_user_origin_tweets(rec_users_tweets, recommended_uid, rem=False), 'user_origin_tweets')
    show_tweets(extract_user_quoted_tweets(rec_users_tweets, recommended_uid, rem=False), 'user_quoted_tweets')
    show_tweets(extract_user_retweeted_tweets(rec_users_tweets, recommended_uid, rem=False), 'user_retweeted_tweets')
    show_tweets(extract_user_replied_tweets(rec_users_tweets, recommended_uid, rem=False), 'user_replied_tweets')



 #################################################################################################### 
 User ID: 1720551038975930369, repetitions: 24
 #################################################################################################### 

----------------------------------------------------------------------------------------------------
Number of user_origin_tweets: 83 

----------------------------------------------------------------------------------------------------
شهدا زنده اند و نزد پروردگار روزی می خورند.
#وداع_با_لالهها
#فرزندان_فاطمه https://t.co/2cLDebDLpY
----------------------------------------------------------------------------------------------------
شهید گمنام یعنی:
شهیدی که میتوانست عقب بیاید اما ماند!
#وداع_با_لالهها
#فرزندان_فاطمه https://t.co/axbOY1BTAY
----------------------------------------------------------------------------------------------------
با شهدا بودن سخت نیست
باشهدا ماندن سخته
#وداع_با_لالهها
#فرزندان_فاطمه https://t.co/NHpktXQZcu
---

In [39]:
end_notebook_t = datetime.datetime.now()
print(end_notebook_t,'\n')

2024-02-02 03:30:34.989209 



In [45]:
print("The total time spent for showing a sample of tweets, count tweets and get users:", str(start_calculating_matrix - start_notebook_t).split('.')[0])
print("The total time spent for calculating Primary Rating Matrix:                     ", str(start_extracting_tweets_id_t - start_calculating_matrix).split('.')[0])
print("The total time spent for extracting Recommended Tweets' ID:                     ", str(start_extract_and_show_tweets_t - start_extracting_tweets_id_t).split('.')[0])
print("The total time spent for extract and show Tweets:                               ", str(start_user_recommending_t - start_extract_and_show_tweets_t).split('.')[0])
print("The total time spent for extracting Recommended Ussers:                         ", str(start_extract_recommended_users_tweets_t - start_user_recommending_t).split('.')[0])
print("The total time spent for extract and show Tweets' of Recommended Users:         ", str(end_extract_recommended_users_tweets_t - start_extract_recommended_users_tweets_t).split('.')[0])
print('_'*90)
print("The total time spent for this script:                                           ", str(end_notebook_t-start_notebook_t).split('.')[0])


The total time spent for showing a sample of tweets, count tweets and get users: 0:01:11
The total time spent for calculating Primary Rating Matrix:                      0:00:38
The total time spent for extracting Recommended Tweets' ID:                      0:00:33
The total time spent for extract and show Tweets:                                0:00:33
The total time spent for extracting Recommended Ussers:                          0:00:33
The total time spent for extract and show Tweets' of Recommended Users:          0:00:24
__________________________________________________________________________________________
The total time spent for this script:                                            0:03:55
